# StateFlow: Build Workflows through State-Oriented Actions

AutoGen offers conversable agents powered by LLM, tool or human, which can be used to perform tasks collectively via automated chat. In this notebook, we introduce the StateFlow feature, which allows you to build workflows with AutoGen agents from a state-oriented perspective.


## Construct a StateFlow model
To construct a StateFlow model, we need to inherit the `StateFlow` class and define the following variables:
- `self.states`: a dictionary of state name and a sequence of actions to be executed in that state.    
An action can be:
    - a function that takes in a list of messages and returns a string or a dict. 
    - an `ConversableAgent` object that has `generate_reply` and `reset` method. `reset` will be called in `StateFlow.run()`.
    - a str tht will be converted to a dict {'content': str, 'role': 'user'} to be appended to context history.
    - a dict following OpenAI message format, will be appended to context history.
- `self.initial_state`: the initial state of the workflow.
- `self.final_states`: a list of states that indicate the end of the workflow. Note there can be multiple final states.
- `self.transitions`: a dictionary of state name and a transition function to determine the next state.
There are two types of transitions:
    - static string matching: Check whether the last message contains a string, for example, "exitcode: 1".
    - use an LLM to determine the next state: Instruct an LLM to determine the current state. For example, whether the problem is solved based on the history (or last message). You still need to instruct the LM to generate responses like "Yes" or "No" and match the response.
- `self.max_transitions`: the maximum number of transitions allowed.

```python
# Example of a simple workflow
class ResearchFlow(stateflow.StateFlow):
    def __init__(self):
        super().__init__()

        # coder, executor, and scientist are AutoGen agents
        self.states = {
            "init": [],
            "retrieve": [coder, executor], # execute code to retrieve papers
            "research": [scientist],
            "end": [],            
        }
        self.initial_state = "init"
        self.final_states = ["end"]
        self.transitions = {
            "init": "retrieve",
            "retrieve": lambda messages: "retrieve" if "exitcode: 1" in messages[-1]["content"] else "research",
            "research": "end",
            "end": "end",
        }
        self.max_transitions = 10
    

    def output_extraction(self, messages):
        """Overwrite this message to extract the output from a list of messages when the workflow ends.
        Default is to return the full list of messages.

        Example:
            - return messages[-1]["content"]
            - return messages[-1]["content"].split("FINAL ANSWER:")[-1].strip()
        """
        pass
```

## Run the StateFlow model
Call `StateFlow.run()` to start the workflow. The method will first reset the states and transitions, including stateful `Agent` in states. Then it executes the workflow until it reaches a final state or the maximum number of transitions is reached. The method returns the output of the workflow.
```python
# Example
output = research.run(task="Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains.", verbose=True)
```


## Requirements
````{=mdx}
:::info Requirements
Install `pyautogen`:
```bash
pip install pyautogen
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-1106-preview"],
    },
)

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/llm_configuration).
:::
````

## Example 1: A simple workflow for research

<!-- ![SF_Example_1](./sf_example_1.png) -->
<figure>
    <img src="./sf_example_1.png"  width="700"
         alt="SF_Example_1">
</figure>

Here is a simple workflow for research:
The Coder will write code to retrieve papers from the internet. The code will be executed by executor. 
If code failed, the code will be revised by the engineer and executed again. 
When the code is executed successfully, the scientist will read the papers and write a summary.


We define the following agents:
- Coder: Retrieve papers from the internet by writing code.
- Executor: Execute the code.
- Scientist: Read the papers and write a summary.








In [2]:
from autogen.agentchat.contrib import stateflow

gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}

coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=gpt4_config,
    system_message="""You are the Coder. Given a topic, write code to retrieve related papers from the arXiv API, print their title, authors, abstract, and link.
You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the Coder and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=gpt4_config,
    system_message="""You are the Scientist. Please categorize papers after seeing their abstracts printed and create a markdown table with Domain, Title, Authors, Summary and Link""",
)


class ResearchFlow(stateflow.StateFlow):
    def __init__(self):
        super().__init__()

        # define states and a sequence of actions in each state
        # an action can be 
        # 1. Dict (in the form of message with key "content" and "role")
        # 2. an Agent class, should have generate_reply and reset method.
        # 3. an function that takes in a list of messages and return a Dict
        self.states = {
            "init": [],
            "retrieve": [coder, executor],
            "research": [scientist],
            "end": [],            
        }
        # define initial and final states
        self.initial_state = "init"
        self.final_states = ["end"]

        # define state transitions
        # either a function that takes in a list of messages and returns the next state, or a string that represents the next state
        self.transitions = {
            "init": "retrieve",
            "retrieve": lambda messages: "retrieve" if "exitcode: 1" in messages[-1]["content"] else "research",
            "research": "end",
            "end": "end",
        }

        # set the maximum number of transitions
        self.max_transitions = 10

    def output_extraction(self, messages):
        # override the post processing method, this method is called after the state machine finishes to generate the final output
        return messages[-1]['content']

In [23]:
research = ResearchFlow()
output = research.run(task="Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains.", verbose=True)

*********State init*********
*********State execute*********
Output 1 (Engineer):
To retrieve related papers from the arXiv API, we can use Python with the `requests` library to send a query to the API and then parse the response to extract the required information. Below is a Python script that does this for the topic "LLM applications" (assuming LLM stands for Large Language Models), and it will retrieve 5 to 10 papers from different domains that were submitted in the last week.

```python
import requests
from datetime import datetime, timedelta
import feedparser

# Define the base URL for the arXiv API
ARXIV_API_URL = "http://export.arxiv.org/api/query?"

# Define the search parameters
search_query = "all:\"LLM applications\""
start = 0
max_results = 10
sort_by = "submittedDate"
sort_order = "descending"

# Calculate the date one week ago from today
one_week_ago = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%dT%H:%M:%SZ')

# Construct the query
query = f"search_query={searc

## Example 2